# TP de Text Mining

In [13]:
import os.path as op
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from glob import glob
import re
from collections import Counter

In [14]:
# Load data
print("Loading dataset")
filenames_neg = sorted(glob(op.join('data', 'imdb1', 'neg', '*.txt'))) # Assumes that data are located in data/imdb1/neg
filenames_pos = sorted(glob(op.join('data', 'imdb1', 'pos', '*.txt'))) # Assumes that data are located in data/imdb1/neg
texts_neg = [open(f).read() for f in filenames_neg]
texts_pos = [open(f).read() for f in filenames_pos]
texts = texts_neg + texts_pos
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

Loading dataset
2000 documents


## 1) Compléter la fonction count_word

In [36]:
def count_words(texts):
    voc = {}
    uniq = set()
    # Determine total number of uniques words in the whole corpus of texts
    for text in texts :
        words = set(re.findall(r"\w+",text.lower()))
        uniq = uniq.union(words)
    counts = np.zeros((len(texts), len(uniq)))
    wordidx = 0
    for textidx,text  in enumerate(texts):
        #print(textidx," / ",len(texts), " documents\r" )
        text = re.findall(r"\w+",text.lower()) # Removes non-words characters and tokenize text
        for word in text:
                # If the word's already in the vocabulary dict, increment the count of the word in the counts matrix
                if word in voc: # for the current document
                    counts[textidx, voc[word]] += 1
                # Else add the new word to the vocabulary dict and set it's index in the counts matrix
                else:
                    voc[word] = wordidx 
                    # increment the count in counts for the current word in the current document
                    counts[textidx, wordidx] += 1 
                    wordidx += 1 # Increment the index for the next new word
    return voc, counts    
        

## Test de la fonction

In [37]:
vocabulary, X = count_words(texts)

In [40]:
print(X.shape)

(2000, 39696)


La fonction marche correctement

## 2) Expliquer la classification obtenue dans le fichier poldata.readme

Voir le PDF

## 3) implémenter le classifieur Bayésien naïf

In [ ]:
class NB(BaseEstimator, ClassifierMixin):
    def __init__(self): # No modification, everything is inherited from BaseEstimator et ClassifierMixin
        pass
    
    def fit(self, X, y):
        vocabulary, counts = count_words(X)
        nDocs = len(counts)
        C = set(y)
        prior = {}
        tct = np.zeros([len(C), len(vocabulary)])
        array_condprob = np.copy(tct)
        cond_prob = {}
        for i,c in enumerate(C):
            # Get lines' index according to the membership of the current class
            ixc = np.where(y == c)[0]
            nDocsInc = len(ixc)
            prior[c] = nDocsInc / nDocs # class frequencies by document
            # Gets all words associated to the current class from counts matrix
            countsByClass = counts[ixc, :] 
            # Sum to get the number of word for each class
            tct[i, :] = np.sum(countsByClass, axis=0)
            # Compute the 
            array_cond_prob[i, :] = (tct[i, :] + 1) / np.sum(tct[i, :] + 1)
            cond_prob[c] = array_cond_prob[i, :]
        self.vocabulary = vocabulary
        self.prior = prior
        self.cond_prob = cond_prob
    
    
    